<a href="https://colab.research.google.com/github/youkiti/oyakudachi/blob/main/pedro_scraper_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PEDroデータベーススクレイパー

このノートブックは、Pedroデータベース（Physiotherapy Evidence Database）から検索結果を取得し、BibTeXファイルとして保存するためのものです。

- **簡単な検索方法**:
  - キーワードを入力するだけで基本検索ができます
  - または高度な検索（Advanced Search）のURLを直接入力することもできます
- PEDroデータベースの検索結果をスクレイピング
- リクエスト間に十分な遅延を実装してサーバー負荷を避ける
- 検索結果のレコードを選択してEndNoteフォーマットでエクスポート
- EndNoteフォーマットをBibTeX（.bib）フォーマットに変換
- ページネーションを処理して検索クエリからすべてのレコードを取得
- JSONレスポンスエラーを適切に処理

このノートブックでは、2つの検索方法を提供しています：

1. **キーワード検索**（初心者向け）:
   - 単純にキーワードを入力するだけで検索できます
   - 例: `lung cancer`、`als`など
   - キーワードは自動的に検索URLに変換されます

2. **高度な検索URL**（上級者向け）:
   - PEDroの[高度な検索ページ](https://search.pedro.org.au/advanced-search)で検索条件を設定
   - 検索実行後のURLをコピー
   - そのURLを直接このノートブックに貼り付け

このスクリプトは個人的な使用目的のみを想定しており、Pedroの公正使用ポリシーに準拠しています：

1. リクエスト間に十分な遅延を実装
2. 特定の検索結果のみをダウンロード（データベース全体ではない）
3. データベースコンテンツの10%未満のダウンロードに制限
4. 急速なリクエストでサーバーに負荷をかけない

**重要な注意事項**: このスクリプトは責任を持って、個人的な研究目的にのみ使用してください。

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import logging
import json
from datetime import datetime
import os
import urllib.parse
from google.colab import files  # Google Colabでファイルをダウンロードするために必要

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)

In [20]:
class PedroScraper:
    """Pedroデータベースから検索結果を取得し、BibTeXファイルとして保存するクラス"""

    def __init__(self, search_url, delay_min=2, delay_max=4, debug=False):
        """初期化"""
        self.search_url = search_url
        self.delay_min = delay_min
        self.delay_max = delay_max
        self.debug = debug
        self.selected_records = []
        self.total_records = 0
        self.base_url = "https://search.pedro.org.au"

        if self.debug:
            logging.getLogger().setLevel(logging.DEBUG)

    def make_request(self, url):
        """指定されたURLにリクエストを送信する"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=30)

            if response.status_code == 200:
                return response
            else:
                logging.error(f"リクエスト失敗: ステータスコード {response.status_code}")
                return None
        except Exception as e:
            logging.error(f"リクエスト中にエラーが発生しました: {str(e)}")
            return None

    def get_total_records(self, soup):
        """検索結果の総レコード数を取得する"""
        try:
            result_text = soup.find(string=lambda s: s and "Found" in s and "records" in s)
            if result_text:
                match = re.search(r'Found\s+(\d+)\s+records', result_text)
                if match:
                    return int(match.group(1))

            result_count_elem = soup.select_one('.result-count')
            if result_count_elem:
                match = re.search(r'(\d+)', result_count_elem.text)
                if match:
                    return int(match.group(1))

            table = soup.select_one('table.search-results')
            if not table:
                table = soup.select_one('table.browse_records')
            if not table:
                table = soup.select_one('table')

            if table:
                rows = table.select('tbody tr')
                if len(rows) > 0:
                    return len(rows)

            return 0
        except Exception as e:
            logging.error(f"総レコード数の取得中にエラーが発生しました: {str(e)}")
            return 0

    def get_pagination_links(self, soup):
        """ページネーションリンクを取得する"""
        pagination_links = []
        try:
            pagination = soup.select('.pagination a')
            for link in pagination:
                href = link.get('href')
                if href and 'page=' in href:
                    full_url = urllib.parse.urljoin(self.base_url, href)
                    pagination_links.append(full_url)
        except Exception as e:
            logging.error(f"ページネーションリンクの取得中にエラーが発生しました: {str(e)}")

        return pagination_links

    def extract_table_records(self, soup):
        """検索結果テーブルからレコード情報を抽出する"""
        records = []

        try:
            table = soup.select_one('table.search-results')
            if not table:
                table = soup.select_one('table.browse_records')
            if not table:
                table = soup.select_one('table')

            if not table:
                logging.error("検索結果テーブルが見つかりませんでした")
                return records

            rows = table.select('tr')

            for row in rows[1:]:
                try:
                    columns = row.select('td')

                    if len(columns) >= 4:  # タイトル、メソッド、スコア、選択ボタンの列があることを確認
                        title_col = columns[0]
                        title_link = title_col.select_one('a')

                        if title_link:
                            title = title_link.text.strip()
                            link = title_link.get('href')
                            if link:
                                full_link = urllib.parse.urljoin(self.base_url, link)
                                record_id = link.split('/')[-1] if '/' in link else None
                            else:
                                full_link = None
                                record_id = None
                        else:
                            title = title_col.text.strip()
                            full_link = None
                            record_id = None

                        method = columns[1].text.strip() if len(columns) > 1 else "N/A"

                        score = columns[2].text.strip() if len(columns) > 2 else "N/A"

                        if record_id and title:
                            records.append({
                                'id': record_id,
                                'title': title,
                                'method': method,
                                'score': score,
                                'url': full_link
                            })
                            logging.debug(f"レコード抽出: ID={record_id}, タイトル={title}, メソッド={method}, スコア={score}")
                except Exception as e:
                    logging.error(f"行の処理中にエラーが発生しました: {str(e)}")
                    continue

        except Exception as e:
            logging.error(f"テーブルレコードの抽出中にエラーが発生しました: {str(e)}")

        return records

    def get_record_details(self, record_id, url):
        """記録の詳細情報を抽出する"""
        response = self.make_request(url)
        if not response:
            logging.error(f"記録 {record_id} の詳細情報の取得に失敗しました")
            return None

        if self.debug:
            with open(f"debug_detail_{record_id}.html", "w", encoding="utf-8") as f:
                f.write(response.text)
            logging.debug(f"詳細ページのHTMLを保存しました: debug_detail_{record_id}.html")

        soup = BeautifulSoup(response.text, 'html.parser')

        try:
            tables = soup.find_all('table', class_='browse_records')
            if not tables:
                tables = soup.select('#search-content table')

            title = ""
            if tables:
                for table in tables:
                    strong_elems = table.select('tr:nth-child(1) td strong')
                    if not strong_elems:
                        strong_elems = table.find_all('strong')

                    if strong_elems and len(strong_elems) > 0:
                        title = strong_elems[0].text.strip()
                        logging.debug(f"テーブル内の強調テキストからタイトルを取得: {title}")
                        break

            if not title:
                h1_elem = soup.select_one('h1.article-title')
                if not h1_elem:
                    h1_elem = soup.select_one('h1')
                if h1_elem:
                    title = h1_elem.text.strip()
                    logging.debug(f"h1要素からタイトルを取得: {title}")

            authors = ""
            if tables:
                for table in tables:
                    author_elems = table.select('tr:nth-child(2) td')
                    if author_elems and len(author_elems) > 0:
                        authors = author_elems[0].text.strip()
                        logging.debug(f"テーブル内の2番目の行から著者情報を取得: {authors}")
                        break

            source = ""
            if tables:
                for table in tables:
                    journal_elems = table.select('tr:nth-child(3) td')
                    if journal_elems and len(journal_elems) > 0:
                        source = journal_elems[0].text.strip()
                        logging.debug(f"テーブル内の3番目の行からジャーナル情報を取得: {source}")
                        break

            abstract = "This record does not have an abstract."

            if tables:
                for table in tables:
                    abstract_elems = table.select('tr:nth-child(5) td p')
                    if abstract_elems and len(abstract_elems) > 0:
                        abstract_text = abstract_elems[0].text.strip()
                        if "Full text" in abstract_text:
                            abstract_text = abstract_text.split("Full text")[0].strip()
                        abstract = abstract_text
                        logging.debug(f"テーブル内の5番目の行から抄録を取得")
                        break

            year = ""
            if source:
                year_match = re.search(r'\b(19|20)\d{2}\b', source)
                if year_match:
                    year = year_match.group(0)

            return {
                'id': record_id,
                'title': title,
                'authors': authors,
                'year': year,
                'source': source,
                'abstract': abstract,
                'url': url
            }

        except Exception as e:
            logging.error(f"記録 {record_id} の詳細情報の抽出中にエラーが発生しました: {str(e)}")
            return None

    def select_record(self, record, table_record=None):
        """記録を選択して保存する"""
        try:
            if not record:
                return False

            if table_record:
                record['method'] = table_record.get('method', 'N/A')
                record['score'] = table_record.get('score', 'N/A')
                if not record.get('title') and table_record.get('title'):
                    record['title'] = table_record.get('title')

            self.selected_records.append(record)
            return True

        except Exception as e:
            logging.error(f"記録の選択中にエラーが発生しました: {str(e)}")
            return False

    def display_progress(self, current, total):
        """進捗状況を表示する"""
        percentage = (current / total) * 100 if total > 0 else 0
        print(f"\r進捗状況: {current}/{total} 件処理済み ({percentage:.1f}%) - 選択済み: {len(self.selected_records)} 件", end="")

    def export_selected_records(self):
        """選択された記録をエクスポートする"""
        if not self.selected_records:
            logging.warning("エクスポートする記録がありません")
            return None

        try:
            now = datetime.now()
            timestamp = now.strftime("%Y%m%d_%H%M%S")

            endnote_file = f"pedro_export.enw"
            with open(endnote_file, "w", encoding="utf-8") as f:
                for record in self.selected_records:
                    f.write("%0 Journal Article\n")

                    if record.get('title'):
                        f.write(f"%T {record['title']}\n")

                    if record.get('authors'):
                        authors = record['authors'].split(',')
                        for author in authors:
                            author = author.strip()
                            if author:
                                f.write(f"%A {author}\n")

                    if record.get('year'):
                        f.write(f"%D {record['year']}\n")

                    if record.get('source'):
                        f.write(f"%J {record['source']}\n")

                    if record.get('abstract'):
                        f.write(f"%X {record['abstract']}\n")

                    if record.get('url'):
                        f.write(f"%U {record['url']}\n")

                    notes = []
                    if record.get('method') and record['method'] != 'N/A':
                        notes.append(f"Method: {record['method']}")
                    if record.get('score') and record['score'] != 'N/A':
                        notes.append(f"Score: {record['score']}")

                    if notes:
                        f.write(f"%Z {'; '.join(notes)}\n")

                    f.write("\n")

            bibtex_file = self.convert_to_bibtex()

            logging.info(f"エクスポートが完了しました: {len(self.selected_records)}件のレコード")

            return {
                "endnote_file": endnote_file,
                "bib_file": bibtex_file
            }

        except Exception as e:
            logging.error(f"記録のエクスポート中にエラーが発生しました: {str(e)}")
            return None

    def convert_to_bibtex(self):
        """選択された記録をBibTeX形式に変換する"""
        if not self.selected_records:
            return None

        try:
            bibtex_file = f"pedro_export.bib"
            with open(bibtex_file, "w", encoding="utf-8") as f:
                for record in self.selected_records:
                    first_author = ""
                    if record.get('authors'):
                        first_author_match = re.search(r'([A-Za-z\-]+)', record['authors'])
                        if first_author_match:
                            first_author = first_author_match.group(1).lower()

                    year = record.get('year', '')

                    bibtex_key = f"{first_author}_{year}_{record['id']}"
                    bibtex_key = re.sub(r'[^a-z0-9_]', '', bibtex_key)

                    f.write(f"@article{{{bibtex_key},\n")

                    if record.get('title'):
                        title = record['title'].replace("{", "\\{").replace("}", "\\}")
                        f.write(f"  title = {{{title}}},\n")

                    if record.get('authors'):
                        authors = record['authors'].replace("{", "\\{").replace("}", "\\}")
                        f.write(f"  author = {{{authors}}},\n")

                    if record.get('year'):
                        f.write(f"  year = {{{record['year']}}},\n")

                    if record.get('source'):
                        journal_match = re.match(r'(.+?)(?:\s+\d{4}|$)', record['source'])
                        if journal_match:
                            journal = journal_match.group(1).strip()
                            f.write(f"  journal = {{{journal}}},\n")
                        else:
                            f.write(f"  journal = {{{record['source']}}},\n")

                    if record.get('abstract'):
                        abstract = record['abstract'].replace("{", "\\{").replace("}", "\\}")
                        f.write(f"  abstract = {{{abstract}}},\n")

                    if record.get('url'):
                        f.write(f"  url = {{{record['url']}}},\n")

                    notes = []
                    if record.get('method') and record['method'] != 'N/A':
                        notes.append(f"Method: {record['method']}")
                    if record.get('score') and record['score'] != 'N/A':
                        notes.append(f"Score: {record['score']}")

                    if notes:
                        note_text = "; ".join(notes)
                        f.write(f"  note = {{{note_text}}},\n")

                    f.write("}\n\n")

            return bibtex_file

        except Exception as e:
            logging.error(f"BibTeXへの変換中にエラーが発生しました: {str(e)}")
            return None

    def scrape(self):
        """検索結果をスクレイピングする"""
        try:
            logging.info(f"検索URLからのスクレイピングを開始: {self.search_url}")

            response = self.make_request(self.search_url)
            if not response:
                return {"success": False, "message": "検索結果の取得に失敗しました"}

            if self.debug:
                with open("debug_page.html", "w", encoding="utf-8") as f:
                    f.write(response.text)
                logging.debug("検索結果ページのHTMLを保存しました: debug_page.html")

            soup = BeautifulSoup(response.text, 'html.parser')

            self.total_records = self.get_total_records(soup)
            logging.info(f"検索結果の総レコード数: {self.total_records}")

            if self.total_records == 0:
                return {"success": False, "message": "検索結果が見つかりませんでした"}

            table_records = self.extract_table_records(soup)
            logging.info(f"検索結果テーブルから {len(table_records)} 件のレコードを抽出しました")

            pagination_links = self.get_pagination_links(soup)
            logging.info(f"ページネーションリンク数: {len(pagination_links)}")

            all_table_records = table_records.copy()

            for page_url in pagination_links:
                logging.info(f"追加ページの処理: {page_url}")

                delay = random.uniform(self.delay_min, self.delay_max)
                logging.info(f"{delay:.2f}秒間待機しています...")
                time.sleep(delay)

                page_response = self.make_request(page_url)
                if not page_response:
                    logging.error(f"ページの取得に失敗しました: {page_url}")
                    continue

                page_soup = BeautifulSoup(page_response.text, 'html.parser')

                page_records = self.extract_table_records(page_soup)
                logging.info(f"ページから {len(page_records)} 件のレコードを抽出しました")

                all_table_records.extend(page_records)

            processed_count = 0

            for table_record in all_table_records:
                record_id = table_record.get('id')
                record_url = table_record.get('url')

                if not record_id or not record_url:
                    logging.warning(f"レコードIDまたはURLが不足しています: {table_record}")
                    continue

                logging.info(f"記録を選択しています: ID={record_id}, タイトル={table_record.get('title')}")

                delay = random.uniform(self.delay_min, self.delay_max)
                logging.info(f"{delay:.2f}秒間待機しています...")
                time.sleep(delay)

                record_details = self.get_record_details(record_id, record_url)

                if record_details:
                    logging.info(f"記録 {record_id} の詳細情報を取得しました: ")
                    self.select_record(record_details, table_record)
                else:
                    logging.warning(f"記録 {record_id} の詳細情報の取得に失敗しました")
                    basic_record = {
                        'id': record_id,
                        'title': table_record.get('title', ''),
                        'authors': '',
                        'year': '',
                        'source': '',
                        'abstract': '',
                        'url': record_url
                    }
                    self.select_record(basic_record, table_record)

                processed_count += 1
                self.display_progress(processed_count, len(all_table_records))

            print()  # 進捗表示の後に改行

            export_result = self.export_selected_records()

            if not export_result:
                return {"success": False, "message": "記録のエクスポートに失敗しました"}

            return {
                "success": True,
                "total_records": len(self.selected_records),
                "endnote_file": export_result["endnote_file"],
                "bib_file": export_result["bib_file"]
            }

        except Exception as e:
            logging.error(f"スクレイピング中にエラーが発生しました: {str(e)}")
            return {"success": False, "message": f"スクレイピング中にエラーが発生しました: {str(e)}"}

In [10]:
def build_search_url(keyword):
    """
    キーワードから検索URLを生成する関数

    引数:
        keyword (str): 検索キーワード

    戻り値:
        str: 検索URL
    """
    encoded_keyword = urllib.parse.quote_plus(keyword)

    base_url = "https://search.pedro.org.au/advanced-search/results"
    search_url = f"{base_url}?abstract_with_title={encoded_keyword}&therapy=0&problem=0&body_part=0&subdiscipline=0&topic=0&method=0&authors_association=&title=&source=&year_of_publication=&date_record_was_created=&nscore=&perpage=20&find=&find=Start+Search"

    return search_url

def is_valid_pedro_url(url):
    """
    URLがPedroの検索結果URLかどうかを確認する関数

    引数:
        url (str): 確認するURL

    戻り値:
        bool: 有効なPedro検索URLならTrue、そうでなければFalse
    """
    if not url.startswith("https://search.pedro.org.au/"):
        return False

    if "/advanced-search/results" not in url and "/search/results" not in url:
        return False

    return True

In [21]:
# 検索方法を選択
search_method = input("検索方法を選択してください（1: キーワード検索、2: 高度な検索URL）: ")

search_url = None

if search_method == "1":
    search_keyword = input("検索キーワードを入力してください（例: lung cancer）: ")
    search_url = build_search_url(search_keyword)
    print(f"生成された検索URL: {search_url}")

elif search_method == "2":
    print("高度な検索URLを入力してください。")
    print("ヒント: https://search.pedro.org.au/advanced-search で検索条件を設定し、検索後のURLをコピーしてください。")
    search_url = input("URL: ")

    if not is_valid_pedro_url(search_url):
        print("警告: 入力されたURLはPedroの検索URLではないようです。")
        proceed = input("それでも続行しますか？ (y/n): ")
        if proceed.lower() != 'y':
            raise ValueError("無効なURLが入力されました。スクリプトを終了します。")
else:
    raise ValueError("無効な選択です。1または2を入力してください。")

scraper = PedroScraper(
    search_url=search_url,
    delay_min=2,
    delay_max=4,
    debug=False
)

result = scraper.scrape()

if result["success"]:
    print(f"\n成功！ {result['total_records']} 件のレコードが取得されました。")
    print(f"EndNote ファイル: {result['endnote_file']}")
    print(f"BibTeX ファイル: {result['bib_file']}")

    files.download(result["bib_file"])
    files.download(result["endnote_file"])

else:
    print(f"\nエラー: {result['message']}")

検索方法を選択してください（1: キーワード検索、2: 高度な検索URL）: 1
検索キーワードを入力してください（例: lung cancer）: als lung
生成された検索URL: https://search.pedro.org.au/advanced-search/results?abstract_with_title=als+lung&therapy=0&problem=0&body_part=0&subdiscipline=0&topic=0&method=0&authors_association=&title=&source=&year_of_publication=&date_record_was_created=&nscore=&perpage=20&find=&find=Start+Search
進捗状況: 8/8 件処理済み (100.0%) - 選択済み: 8 件

成功！ 8 件のレコードが取得されました。
EndNote ファイル: pedro_export.enw
BibTeX ファイル: pedro_export.bib


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>